In [ ]:
# import stuff
import random

random.seed(0)

import pickle

import networkx as nx
import numpy as np

np.random.seed(0)

import os

# Load all libraries
import pickle

import matplotlib
import matplotlib.pyplot as plt
import motile
import networkx as nx
import numpy as np
import pandas as pd
import pymeshfix
import seaborn as sns
import skimage
import zarr
from joblib import Parallel, delayed
from matplotlib import pyplot as plt
from matplotlib.colors import LightSource
from motile.plot import draw_solution, draw_track_graph
from skimage.io import imread, imsave
from skimage.measure import marching_cubes
from skimage.transform import rescale
from tqdm import tqdm
from trimesh import Trimesh
from trimesh.smoothing import filter_taubin

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
channel = "GFP"
import seaborn as sns

pallette = {"Matrigel": "#17ad97", "No Matrix": "#4d4d4d", "Agarose": "#98d9d1"}
sns.set_context("paper", rc={"font.size": 8, "axes.titlesize": 8, "axes.labelsize": 8})
rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.size": 5,
}
plt.rcParams.update(rc)
cm = 1 / 2.54  # centimeters in inches

In [ ]:
positions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
matrigel = [1, 2, 3, 4]
no_matrix = [5, 6, 7, 8, 13, 14, 15, 16]
agar = [9, 10, 11, 12]
all_dfs = []
smootheness = 5
label = f"lumen_masks_smooth_{smootheness}_processed"
for position in tqdm(positions):
    try:
        track_path = f"/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking/tracks/Position_{position}_Settings_1_Processed/"
        track_graph = pickle.load(
            open(track_path + f"{channel}_{label}_track_motile_graph.pickle", "rb")
        )
        nx_graph = pickle.load(
            open(track_path + f"{channel}_{label}_track.pickle", "rb")
        )
        adjacency_network = nx.to_scipy_sparse_array(nx_graph)
        df = []
        for t in range(125):
            nodes = track_graph.nodes_by_frame(t)
            n_nodes = len(nodes)
            if n_nodes == 0:
                n_nodes = 1
            node_loc = np.where(
                np.array([(node in nodes) for node in list(nx_graph.nodes)])
            )[0]
            n_divisions = np.count_nonzero(
                adjacency_network[node_loc].sum(1) == 2
            )  # /n_nodes
            n_fusions = np.count_nonzero(
                adjacency_network[:, node_loc].sum(0) == 2
            )  # /n_nodes
            df.append([t, n_divisions, n_fusions, len(nodes), position])
        df = pd.DataFrame(df)
        df.columns = ["Hour", "N divisions", "N fusions", "N lumen", "position"]
        df["N divisions RM"] = df["N divisions"].rolling(10, min_periods=1).mean()
        df["Smoothened number of fusions"] = (
            df["N fusions"].rolling(10, min_periods=1).mean()
        )
        df["Fusions per lumen"] = df["N fusions"] / df["N lumen"]
        df["Smoothened fusions per lumen"] = (
            df["Fusions per lumen"].rolling(25, min_periods=1).mean()
        )

        df["Number of lumen"] = df["N lumen"]
        if position in matrigel:
            df["Condition"] = "Matrigel"
        if position in no_matrix:
            df["Condition"] = "No Matrix"
        if position in agar:
            df["Condition"] = "Agarose"

        all_dfs.append(df)
    except:
        print(f"Position {position} still running")
all_dfs = pd.concat(all_dfs)
all_dfs.index = np.arange(len(all_dfs))
all_dfs["Day"] = (all_dfs["Hour"] / 24) + 4
all_dfs["Smoothened number  \n of fusions"] = all_dfs["Smoothened number of fusions"]
sub_df = all_dfs[["Hour", "Smoothened number  \n of fusions", "Number of lumen"]]

In [ ]:
fig, ax = plt.subplots(figsize=(5 * cm, 5 * cm))
sns.despine(left=True, bottom=True, right=True)
sns.lineplot(
    x="Day",
    y="Smoothened number  \n of fusions",
    data=all_dfs,
    hue="Condition",
    palette=pallette,
    ci="sd",
)
for condition in all_dfs["Condition"].unique():
    condition_data = all_dfs[all_dfs["Condition"] == condition]
    max_day = (
        condition_data.groupby("Day")["Smoothened number  \n of fusions"]
        .mean()
        .idxmax()
    )
    plt.axvline(x=max_day, color=pallette[condition], alpha=0.7, linestyle="-.")

sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
fig, ax = plt.subplots(figsize=(5 * cm, 5 * cm))
sns.despine(left=True, bottom=True, right=True)
sns.lineplot(
    x="Day",
    y="Number of lumen",
    data=all_dfs,
    hue="Condition",
    palette=pallette,
    ci="sd",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))